# Tokenization Preprocessing for BERT-style Models

In transformer models such as BERT and DistilBERT, tokenization is only the first part of a larger **input preprocessing pipeline**. After tokenization, several additional structures are created so that the model can correctly interpret the input sequence during training and inference.

## From Text to Model Input

Consider a short input sentence. The preprocessing pipeline transforms it through several stages so that it can be consumed by the transformer encoder. These stages are deterministic and defined by the model architecture.

The overall flow is:

raw text → tokens → special tokens → token IDs → padding → attention masks → token type IDs

Each step adds structure and constraints that the transformer relies on.

## Special Tokens: CLS and SEP

Transformer encoders based on BERT introduce special tokens that do not correspond to words in the input text.

The **CLS token** is inserted at the beginning of every input sequence. It serves as a global representation of the entire sequence. During training, the hidden state corresponding to this token is commonly used for sequence-level tasks such as classification or regression. Conceptually, CLS signals to the model that a new sequence has started and that a summary representation will be required.

The **SEP token** marks boundaries between segments. It is used to separate sentences or to signal the end of a sequence. When a single sentence is used, a SEP token is still added to clearly delimit the input.

These tokens are part of the model’s fixed vocabulary and are treated like any other token during embedding and attention.

## Token IDs

After adding special tokens, each token is mapped to a unique integer from the model’s vocabulary. These integers are called **token IDs**. Token IDs are indices into the embedding matrix and are the actual numerical inputs to the model.

At this stage, the input is a sequence of integers of variable length.

## Fixed-Length Inputs and Padding

Transformer models expect inputs of a fixed length. This is necessary for efficient batching and consistent tensor shapes.

If the token sequence is shorter than the chosen maximum length, padding is applied. Padding consists of appending a special value (typically zero) until the sequence reaches the required length. If the sequence is longer than the maximum length, it is truncated.

Padding does not represent meaningful information. Its only purpose is to enforce a uniform input shape across different samples.

## Attention Masks

Because padding tokens carry no semantic meaning, the model must be told to ignore them. This is done using an **attention mask**.

The attention mask is a binary sequence with the same length as the input:

- A value of 1 indicates a real token that should be attended to.
- A value of 0 indicates padding that should be ignored.

During self-attention, these masks prevent padded positions from influencing the attention scores.

## Token Type IDs (Segment IDs)

BERT-style models can process pairs of sentences simultaneously. To distinguish between different segments, **token type IDs** are used.

Token type IDs assign an integer label to each token indicating which sentence it belongs to. For example:

- Tokens from the first sentence are assigned one label.
- Tokens from the second sentence are assigned a different label.

When only a single sentence is provided, all tokens receive the same token type ID. Padding positions also receive a default value, since they do not belong to any sentence.

Token type IDs allow the model to learn relationships between sentence pairs, such as entailment or similarity.

## Single-Sequence Input Structure

For a single sentence, the model input consists of:

- Token IDs representing tokens and special tokens
- Attention masks indicating real tokens versus padding
- Token type IDs indicating a single segment

Even though token type IDs may seem redundant in this case, they are still generated for architectural consistency.

## Two-Sequence Input Structure

When two sentences are provided together, the input structure changes slightly:

- A separator token marks the boundary between sentences
- Token type IDs switch values at the sentence boundary
- Attention masks still indicate valid tokens versus padding

This allows the model to jointly encode both sentences while still preserving their identity.

## Why All Steps Matter

Although the model ultimately consumes token IDs, attention masks, and token type IDs, each preprocessing step depends on the previous one. Padding must be defined before masks can be created, and sentence boundaries must be identified before token type IDs can be assigned.

These preprocessing steps ensure that the transformer encoder receives well-structured, interpretable inputs and behaves consistently during training.


In [1]:
import transformers
from transformers import DistilBertTokenizer

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

### Single Sentence

In [3]:
inputs = tokenizer.encode_plus(
    "I love mathematics!",          # Raw input text to be tokenized and encoded

    add_special_tokens = True,      # Adds special tokens such as [CLS] at the beginning
                                    # and [SEP] at the end of the sequence

    max_length = 20,                # Forces the final sequence to have a fixed length
                                    # (after tokenization and adding special tokens)

    padding = 'max_length',         # Pads the sequence with zeros if it is shorter
                                    # than max_length

    truncation = True,              # Truncates the sequence if it exceeds max_length

    return_token_type_ids = True,   # Returns token type IDs (segment IDs)
                                    # Used to distinguish multiple sentences

    return_attention_mask = True    # Returns the attention mask
                                    # 1 for real tokens, 0 for padding
)


In [4]:
print(f"Input IDs: {inputs['input_ids']}")
print(f"Attention Mask: {inputs['attention_mask']}")
print(f"Token type ids: {inputs['token_type_ids']}")

Input IDs: [101, 1045, 2293, 5597, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention Mask: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Token type ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Two Sentences

In [5]:
inputs = tokenizer.encode_plus(
    "I love mathematics!",
    "Linear algebra is at the core of machine learning",
    add_special_tokens = True,
    max_length = 20,
    padding = 'max_length',
    truncation = True,
    return_token_type_ids = True,
    return_attention_mask = True
)

In [6]:
print(f"Input IDs: {inputs['input_ids']}")
print(f"Attention Mask: {inputs['attention_mask']}")
print(f"Token type ids: {inputs['token_type_ids']}")

Input IDs: [101, 1045, 2293, 5597, 999, 102, 7399, 11208, 2003, 2012, 1996, 4563, 1997, 3698, 4083, 102, 0, 0, 0, 0]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
Token type ids: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
